In [1]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = 'python'
import numpy as np

from d3rlpy.algos import DiscreteCQL

from recsys_mdp.recsys_mdp import RecSysMDP
from recsys_mdp.utils import to_d3rlpy_form
from recsys_mdp.d3rlpy_recsys_metrics import evaluate_in_positive, \
                                            evaluate_in_negative

In [2]:
mdp_train = RecSysMDP(load_from_file=True, path='./data/ml_100k_first_1000_train_90')
mdp_test = RecSysMDP(load_from_file=True, path='./data/ml_100k_first_1000_test_74')
#
dataset_train = to_d3rlpy_form(mdp_train.states, mdp_train.rewards,
                               mdp_train.actions, mdp_train.termations)

dataset_test = to_d3rlpy_form(mdp_test.states, mdp_test.rewards,
                               mdp_test.actions, mdp_test.termations)

Data loaded!
Data loaded!


In [3]:
algo = DiscreteCQL(use_gpu=True)

In [4]:
algo.build_with_dataset(dataset_train)
algo.load_model('CQL.pt')

In [5]:
dataset_test[0].observations[0][:5]

array([0.8619744 , 0.5426975 , 0.30939713, 0.85273725, 0.20685284],
      dtype=float32)

In [6]:
item_mapping = mdp_train.item_mapping
user_mapping = mdp_train.user_mapping
inv_user_mapping = mdp_train.inv_user_mapping

In [7]:
algo.predict([dataset_test[0].observations[0]])

array([212])

In [8]:
def step_by_step_top(new_observation, emb_size, top_k, item_mapping):
    for i in range(top_k):
        actions = algo.predict([new_observation])
        action_embedding = item_mapping[actions[0]]
       # print(action_embedding in new_observation)
        idx = new_observation[-emb_size:]
        new_observation = np.append(new_observation[emb_size:-emb_size], action_embedding, axis=0)
        new_observation = np.append(new_observation, idx, axis=0)
        
        
      #  print(new_observation[:10])
       # print(new_observation[-10:])
        yield actions[0]

In [10]:
framestack = 5
use_user_emb = 1
emb_size = 64
num_of_epiode = 1950
generator = step_by_step_top(dataset_train[num_of_epiode].observations[5], 64, 10, item_mapping)
pkey = dataset_train[num_of_epiode].observations[0].reshape(framestack + use_user_emb, emb_size)[-1]
for okey in inv_user_mapping:
    if np.mean(np.abs(np.asarray(pkey) - np.asarray(okey))) < 0.1:
        key = okey
user_idx = inv_user_mapping[tuple(key)]

In [11]:
df = mdp_train.dataframe
items_of_user = df[df['user_idx'] == user_idx]['item_idx']
#print(items_of_user)

In [12]:
user_log = df[df['user_idx'] == user_idx]
for i in generator:
    if i in items_of_user:
        print("IN ",  i)
        print(user_log[user_log['item_idx'] == i]['rating'].values)
    else:
         print("----------OUT ", i)

----------OUT  212
----------OUT  212
----------OUT  212
----------OUT  212
----------OUT  212
----------OUT  212
----------OUT  212
----------OUT  212
----------OUT  212
----------OUT  212


In [13]:
framestack = 5
use_user_emb = 1
emb_size = 64
num_of_epiode = 0

for num_of_epiode in range(0,5000, 150):
    generator = step_by_step_top(dataset_train[num_of_epiode].observations[5], 64, 10, item_mapping)
    pkey = dataset_train[num_of_epiode].observations[0].reshape(framestack + use_user_emb, emb_size)[-1]
    for okey in inv_user_mapping:
        if np.mean(np.abs(np.asarray(pkey) - np.asarray(okey))) < 0.1:
            key = okey
    user_idx = inv_user_mapping[tuple(key)]
    df = mdp_train.dataframe
    items_of_user = df[df['user_idx'] == user_idx]['item_idx']
    user_log = df[df['user_idx'] == user_idx]
    for i in generator:
        if i in items_of_user:
            print("IN ",  i)
            print(user_log[user_log['item_idx'] == i]['rating'].values)
#         else:
#              print("----------OUT ", i)

IN  212
[3.]
IN  212
[3.]
IN  212
[3.]
IN  212
[3.]
IN  212
[3.]
IN  212
[3.]
IN  212
[3.]
IN  212
[3.]
IN  212
[3.]
IN  212
[3.]


IndexError: list index out of range

In [331]:
len(dataset_train)

5200

In [13]:

dataset_train[num_of_epiode].observations[0][:64]

array([0.5063616 , 0.84323156, 0.15037137, 0.02437739, 0.46300983,
       0.07075968, 0.10637806, 0.35955566, 0.36580744, 0.36928186,
       0.9395704 , 0.46419388, 0.6277259 , 0.16913533, 0.7779768 ,
       0.71310896, 0.7861116 , 0.3630307 , 0.7998114 , 0.58007914,
       0.6490179 , 0.9313    , 0.16207871, 0.39254352, 0.96539474,
       0.50316805, 0.20591237, 0.73959893, 0.8077247 , 0.14521034,
       0.6526091 , 0.2828908 , 0.634223  , 0.63383204, 0.51470906,
       0.5891453 , 0.5109552 , 0.89045566, 0.25270388, 0.27987373,
       0.35095105, 0.41026044, 0.4526716 , 0.5976873 , 0.40879905,
       0.9995152 , 0.53424555, 0.8711585 , 0.10440695, 0.20928983,
       0.67888963, 0.55994135, 0.36137995, 0.81349474, 0.3208876 ,
       0.5156317 , 0.0096271 , 0.34902433, 0.908778  , 0.79926336,
       0.22688009, 0.71755666, 0.6385972 , 0.54383695], dtype=float32)

In [14]:
dataset_train[num_of_epiode].observations[0][128+128:128+128+64]

array([0.37687427, 0.5196445 , 0.60941964, 0.9279342 , 0.30972305,
       0.53416365, 0.6269171 , 0.8819466 , 0.67475337, 0.2338695 ,
       0.45984766, 0.6330028 , 0.5384787 , 0.40182602, 0.75296295,
       0.64037824, 0.00234411, 0.68452555, 0.84262246, 0.45729417,
       0.5991173 , 0.49228576, 0.41566038, 0.8686623 , 0.34751663,
       0.35995212, 0.94489   , 0.55298185, 0.02874127, 0.40183535,
       0.26483846, 0.58645535, 0.398687  , 0.2896845 , 0.7122024 ,
       0.0617929 , 0.5249928 , 0.3157501 , 0.02001354, 0.6538085 ,
       0.26967734, 0.26175678, 0.05694924, 0.7318533 , 0.06407281,
       0.11246518, 0.23374292, 0.16564567, 0.6419434 , 0.94882864,
       0.01491577, 0.8637892 , 0.5918122 , 0.64596766, 0.06548883,
       0.17820129, 0.05943775, 0.6544852 , 0.26445168, 0.35675135,
       0.08829141, 0.34504145, 0.864374  , 0.5673151 ], dtype=float32)

In [130]:
import rs_datasets
ml = rs_datasets.MovieLens()
raitings = ml.ratings

In [131]:
keys = list(set(raitings['item_id']))
item_mapping = dict(zip(keys, list(range(len(keys)))))
raitings['items_idx'] = raitings['item_id'].apply(lambda x: item_mapping[x])
keys = list(set(raitings['user_id']))
item_mapping = dict(zip(keys, list(range(len(keys)))))
raitings['user_idx'] = raitings['user_id'].apply(lambda x: item_mapping[x])

In [133]:
raitings

,user_id,item_id,rating,timestamp,items_idx,user_idx
0,1,1,4.0,964982703,0,0
1,1,3,4.0,964981247,2,0
2,1,6,4.0,964982224,5,0
3,1,47,5.0,964983815,46,0
4,1,50,5.0,964982931,49,0
...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,2291,609
100832,610,168248,5.0,1493850091,3714,609
100833,610,168250,5.0,1494273047,3716,609
100834,610,168252,5.0,1493846352,3718,609
